In [ ]:
!which python

In [ ]:
!which pip

In [ ]:
# Install first all dependencies
!pip install torch torchvision numpy pandas scikit-learn matplotlib skorch opencv-python

In [ ]:
#Import all needed modules
import os
import random
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from shutil import copy2
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Define path and load data
image_dir = 'gt/images'
label_dir = 'gt/labels'
output_dirs = {
    'train': 'split/train',
    'val': 'split/val',
    'test': 'split/test'
}

# Ensure output directories exist
for path in output_dirs.values():
    os.makedirs(path, exist_ok=True)

# List all images and corresponding label files
image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
label_files = [f for f in os.listdir(label_dir) if f.endswith('.txt')]

# Sort to make sure images and labels are matched
image_files.sort()
label_files.sort()

# Check if we have a matching number of image and label files
assert len(image_files) == len(label_files), "Mismatch between number of images and labels"


In [ ]:
# Load Label and create DataFrame
def read_labels(label_file):
    with open(os.path.join(label_dir, label_file), 'r') as f:
        labels = list(map(int, f.read().strip().split(',')))
    return labels

data = []
for img_file, lbl_file in zip(image_files, label_files):
    labels = read_labels(lbl_file)
    data.append({'image': img_file, 'labels': labels})

df = pd.DataFrame(data)

In [ ]:
# Split data
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

# Check the number of samples in each split
print(f"Training samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")
print(f"Test samples: {len(test_df)}")

In [ ]:
# Print sample of data frame 
print(df.head(4))

In [ ]:
# Copy files to corresponding directories
def copy_files(df, split):
    for _, row in df.iterrows():
        img_path = os.path.join(image_dir, row['image'])
        dest_path = os.path.join(output_dirs[split], row['image'])
        copy2(img_path, dest_path)

# Copy files for each split
copy_files(train_df, 'train')
copy_files(val_df, 'val')
copy_files(test_df, 'test')

In [ ]:
# Verify data preparation
# Function to count files in directories
def count_files(dir_path):
    return len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])

# Verify counts
print(f"Training images: {count_files(output_dirs['train'])}")
print(f"Validation images: {count_files(output_dirs['val'])}")
print(f"Test images: {count_files(output_dirs['test'])}")

In [ ]:
# Show resized image
random_file = random.choice(os.listdir(output_dirs['train']))
image = cv2.imread(os.path.join("split", "train", random_file))
resized =  cv2.cvtColor(cv2.resize(image, (224, 224)), cv2.COLOR_BGR2RGB)
plt.imshow(resized)
plt.axis('off')
plt.show()

In [ ]:
# Define data set class
class CustomDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.dataframe.iloc[idx, 0])
        image = Image.open(img_name).convert('RGB')
        labels = np.array(self.dataframe.iloc[idx, 1:].values[0])
        labels = torch.tensor(labels, dtype=torch.float32)

        # Squeeze labels to ensure the correct shape [num_outputs]
        labels = labels.squeeze()
        
        if self.transform:
            image = self.transform(image)
        
        return image, labels

# Define transformations for the training and validation data
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to fit ResNet input
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Adjust brightness, contrast, etc.
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Random affine transformation with translation
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet stats
])

# Create datasets
train_dataset = CustomDataset(dataframe=train_df, image_dir=output_dirs['train'], transform=transform)
val_dataset = CustomDataset(dataframe=val_df, image_dir=output_dirs['val'], transform=transform)
test_dataset = CustomDataset(dataframe=test_df, image_dir=output_dirs['test'], transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Modified ResNet model
class ResNetWithSigmoid(nn.Module):
    def __init__(self, num_outputs = 15):
        super(ResNetWithSigmoid, self).__init__()
        self.base_model = models.resnet50(pretrained=True)
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, num_outputs),
            nn.Sigmoid()  # Ensure the outputs are between 0 and 1
        )
    
    def forward(self, x):
        return self.base_model(x)

In [ ]:
# Prepare model
model = ResNetWithSigmoid()

# Define the parameter grid
# params = {
#     'lr': [0.001, 0.01, 0.1],
#     'max_epochs': [30],
#     'optimizer_weight_decay': [0.01, 0.001, 0.0001],
#     'batch_size': [8, 16, 32],
# }

# Move model to GPU if available
print(torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Train Model with early stopping
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, patience=5):
    train_losses = []
    val_losses = []
    val_accuracies = []
    
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            
            outputs = model(inputs)
            print(outputs[0].shape)
            print(outputs[0])
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * inputs.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        train_losses.append(epoch_loss)
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item() * inputs.size(0)
                val_preds.extend(torch.sigmoid(outputs).cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
        
        epoch_val_loss = val_loss / len(val_loader.dataset)
        epoch_val_accuracy = np.mean(np.equal((np.array(val_preds) > 0.5).astype(int), np.array(val_labels)))
        val_losses.append(epoch_val_loss)
        val_accuracies.append(epoch_val_accuracy)
                
        # Early stopping
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            epochs_without_improvement = 0
            # Save the best model
            torch.save(model.state_dict(), 'model/best_model.pth')
        else:
            epochs_without_improvement += 1
        
        if epochs_without_improvement >= patience:
            print(f'Early stopping triggered after {epoch + 1} epochs without improvement')
            break
        
        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Training Loss: {epoch_loss:.4f}, "
              f"Validation Loss: {epoch_val_loss:.4f}, "
              f"Validation Accuracy: {epoch_val_accuracy:.4f}")
        
    
    return train_losses, val_losses, val_accuracies

# Train the model with early stopping
train_losses, val_losses, val_accuracies = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=50, patience=10)


In [ ]:
# Plot training history
def plot_training_history(train_losses, val_losses, val_accuracies):
    epochs = range(1, len(train_losses) + 1)
    
    plt.figure(figsize=(12, 6))
    
    # Plot training and validation loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'bo-', label='Training Loss')
    plt.plot(epochs, val_losses, 'ro-', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    # Plot validation accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, val_accuracies, 'go-', label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.show()

# Plot training history
plot_training_history(train_losses, val_losses, val_accuracies)

In [ ]:
# Evaluate model
def evaluate_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    test_preds = []
    test_labels = []
    
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            test_loss += loss.item() * inputs.size(0)
            test_preds.extend(torch.sigmoid(outputs).cpu().numpy())
            test_labels.extend(labels.cpu().numpy())
    
    test_loss = test_loss / len(test_loader.dataset)
    test_accuracy = np.mean(np.equal((np.array(test_preds) > 0.5).astype(int), np.array(test_labels)))
    
    print(f'Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}')

# Evaluate the model
evaluate_model(model, test_loader, criterion)